# PCS‑HELIO v4.3 — 06 · EEG × KEC
Band‑power association with KEC metrics; saves coeff table and F3 figure.

In [ ]:
from pathlib import Path; import pandas as pd, numpy as np, json, sys
# Robust import of shared fragments regardless of CWD
ROOT = Path.cwd()
if (ROOT/'notebooks'/'_fragments.py').exists():
    sys.path.insert(0, str(ROOT))
elif (ROOT.parent/'notebooks'/'_fragments.py').exists():
    sys.path.insert(0, str(ROOT.parent))
try:
    from notebooks._fragments import apply_style, preflight_checks, print_contract, qa_assertions, save_manifest
except Exception as e:
    print('[preflight] Failed importing notebooks._fragments:', e)
    def apply_style(): pass
    def preflight_checks(): pass
    def print_contract(): pass
    def qa_assertions(df, rules): pass
    def save_manifest(path, payload): Path(path).parent.mkdir(parents=True, exist_ok=True); Path(path).write_text(json.dumps(payload, indent=2))
from pcs_toolbox.analysis import fit_ols_clustered
apply_style(); preflight_checks(); print_contract()
BASE=Path('.') ; DATA=BASE/'data' ; PROC=DATA/'processed' ; RPTS=BASE/'reports' ; FIG=BASE/'figures'/'metrics'
PROC.mkdir(parents=True, exist_ok=True); RPTS.mkdir(parents=True, exist_ok=True); FIG.mkdir(parents=True, exist_ok=True)


In [ ]:
# Load merged dataset if available; fallback to ZuCo
df = pd.read_csv(PROC/'zuco_kec_merged.csv') if (PROC/'zuco_kec_merged.csv').exists() else (pd.read_csv(PROC/'zuco_aligned.csv') if (PROC/'zuco_aligned.csv').exists() else pd.DataFrame())
# Map EEG power name variants
rename = {'ThetaPower':'theta1','AlphaPower':'alpha1','BetaPower':'beta1','GammaPower':'gamma1'}
for k,v in rename.items():
    if k in df.columns and v not in df.columns:
        df[v] = df[k]
bands = [c for c in ['theta1','alpha1','beta1','gamma1'] if c in df.columns]
print('Bands available:', bands)
# EEG coverage
if bands:
    cov = df[bands].notna().any(axis=1).mean()*100
    if cov < 60:
        print(f'[warn] EEG coverage low: {cov:.1f}%')
# Correlation matrix for quick sanity
cols = [*bands] + [c for c in ['entropy','curvature','coherence'] if c in df.columns]
corr = (df[cols].corr() if cols else pd.DataFrame())
print(corr.round(3) if not corr.empty else '[note] no corr')


In [ ]:
# Simple band-wise OLS vs KEC predictors (clustered by Subject)
results = []
preds = [c for c in ['entropy','curvature','coherence'] if c in df.columns]
if bands and preds and 'Subject' in df.columns:
    for b in bands:
        y = b
        rhs = ' + '.join(preds)
        formula = f"{y} ~ " + rhs
        try:
            m = fit_ols_clustered(df.dropna(subset=[y]+preds), formula, cluster='Subject')
            for term, est in m.params.items():
                if term=='Intercept': continue
                results.append({'band': y, 'term': term, 'estimate': float(est), 'p': float(m.pvalues.get(term, np.nan))})
        except Exception as e:
            print('[warn] OLS failed for', y, e)
coef_df = pd.DataFrame(results) if results else pd.DataFrame()
print(coef_df.head())
# Save coeffs
if not coef_df.empty:
    out_csv = PROC/'models_eeg_coeffs.csv'
    coef_df.to_csv(out_csv, index=False)
    save_manifest(RPTS/'models_eeg_manifest.json', {'rows': int(len(coef_df)), 'bands': sorted(coef_df['band'].unique())})


In [ ]:
# Figure F3: simple heatmap of correlations (if available)
import matplotlib.pyplot as plt
if 'corr' in globals() and hasattr(corr, 'values') and corr.size>0:
    fig, ax = plt.subplots(figsize=(4,3))
    im = ax.imshow(corr.values, cmap='coolwarm', vmin=-1, vmax=1)
    ax.set_xticks(range(len(corr.columns))); ax.set_xticklabels(list(corr.columns), rotation=45, ha='right')
    ax.set_yticks(range(len(corr.index))); ax.set_yticklabels(list(corr.index))
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout(); fig.savefig(FIG/'F3_eeg_corr.png', dpi=150); plt.close(fig)
else:
    print('[note] No correlation computed (insufficient columns).')
# QA
if not coef_df.empty:
    qa_assertions(coef_df, {'min_rows': 3})
